In [1]:
import summary as msy
# my self-make recommend system package
# make sure you put summary.py in the same dictionary with this jupyter notbook
import numpy as np
import pandas as pd
import importlib
from sklearn.metrics.pairwise import cosine_similarity


Using device: cpu


[nltk_data] Downloading package punkt to /Users/lwk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/lwk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
'''this line is used in test for reload the summary.py'''
# importlib.reload(msy)


Using device: cpu


[nltk_data] Downloading package punkt to /Users/lwk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/lwk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'summary' from '/Users/lwk/Documents/web_science/web_science/summary.py'>

### load dataset

In [3]:
df_1, df_2 = msy.ini('Software_5.json', 'meta_Software.json')


# Part 1 Preprocess

initial the class for preprocess

In [4]:
preprocess = msy.Preprocess(df_1, df_2)


Overall description for training set group by user ID

In [4]:
training_des = preprocess.training_data.groupby(['reviewerID']).count()
training_des.overall.describe()


count    1824.000000
mean        5.576206
std         3.488828
min         1.000000
25%         4.000000
50%         5.000000
75%         6.000000
max        51.000000
Name: overall, dtype: float64

Overall description for test set group by user ID

In [5]:
test_des = preprocess.test_data.groupby(['reviewerID']).count()
test_des.overall.describe()


count    1711.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: overall, dtype: float64

Overall description for training set group by asin

In [6]:
trainingit_des = preprocess.training_data.groupby(['asin']).count()
trainingit_des.overall.describe()


count    800.000000
mean      12.713750
std       16.234801
min        1.000000
25%        5.000000
50%        9.500000
75%       17.000000
max      226.000000
Name: overall, dtype: float64

Show the most popular top5 items, and the items with top5 hightest mean rate

In [7]:
trainingit_head = preprocess.training_data.groupby(
    ['asin']).size().reset_index(name='count')
trainingit_head['mean rate'] = list(
    preprocess.training_data.groupby(['asin'])['overall'].mean())
print(trainingit_head.sort_values(['count'], ascending=False)
      .head(5))
print(trainingit_head.sort_values(['mean rate'], ascending=False)
      .head(5))


           asin  count  mean rate
50   B0001FS9NE    226   4.929204
18   B000050ZRE    226   4.929204
41   B0000AZJY6    224   4.928571
560  B00EZPXYP4     91   4.384615
569  B00F8K9MZQ     66   4.318182
           asin  count  mean rate
214  B0015CA9VW      1        5.0
217  B0015T5Q86      3        5.0
213  B0014ZQMXO      2        5.0
206  B0014A2LJ8      1        5.0
191  B0013MCZRK      2        5.0


# Part 2 Collaborative filtering recommendation

initial the class for Collaborative filtering recommendation

In [5]:
coll_recommend = msy.Collaborativefiltering_recommendation(reuse=preprocess)


 3-fold cross-validation

In [6]:
gs_knn = coll_recommend.gridsearch('knn')
gs_svd = coll_recommend.gridsearch('svd')


Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the msd similarity matrix...
Computing the pearson similarity matrix...
Computing the cosine similarity matrix...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the pearson similarity matrix...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matri

show the best parameter and RMSE score.
<font color=red>Warning</font>
In SVD probably due to the small gap between parameters (800, 50) and parameters (100, 10), when you try to rerun the code it is possible that the grid experiment will give another set of parameters as optimal

In [7]:
print(gs_knn.best_score)
print(gs_knn.best_params['rmse'])
print(gs_svd.best_score)
print(gs_svd.best_params['rmse'])


{'rmse': 1.2111196043039298}
{'k': 5, 'sim_options': {'name': 'pearson', 'user_based': True}}
{'rmse': 1.1417805477603624}
{'n_epochs': 800, 'n_factors': 50, 'random_state': 0}


set svd and knn with best parameter and fit them with the training data 

In [7]:
knn_est, knn_model = coll_recommend.knn(
    k=5, sim_options={'name': 'pearson', 'user_based': True})
svd_est, svd_model = coll_recommend.svd(
    n_epochs=800, n_factors=50, random_state=0)


Computing the pearson similarity matrix...
Done computing similarity matrix.


## Part 3 Evaluation of Recommender Systems

initial the class for evaluation

In [8]:
knn_evalue = msy.Evaluation(knn_est, reuse=preprocess)
svd_evalue = msy.Evaluation(svd_est, reuse=preprocess)


MP@k, MAP@k, MRR@k, MHR@k for knn model

In [10]:
print('knn mpk+  {:.6f}'.format(knn_evalue.mean_k(5, 'p_k')))
print('knn mapk+ {:.6f}'.format(knn_evalue.mean_k(5, 'ap_k')))
print('knn mrrk+ {:.6f}'.format(knn_evalue.mean_k(5, 'rr_k')))
print('knn hrk+  {:.6f}'.format(knn_evalue.mean_k(5, 'hr_k')))


knn mpk+  0.003741
knn mapk+ 0.008816
knn mrrk+ 0.008816
knn hrk+  0.018703


MP@k, MAP@k, MRR@k, MHR@k for svd model

In [11]:
print('svd mpk+  {:.6f}'.format(svd_evalue.mean_k(5, 'p_k')))
print('svd mapk+ {:.6f}'.format(svd_evalue.mean_k(5, 'ap_k')))
print('svd mrrk+ {:.6f}'.format(svd_evalue.mean_k(5, 'rr_k')))
print('svd hrk+  {:.6f}'.format(svd_evalue.mean_k(5, 'hr_k')))


svd mpk+  0.001870
svd mapk+ 0.003078
svd mrrk+ 0.003078
svd hrk+  0.009351


rsme for knn and svd model

In [12]:
print('knn rmse+ {}'.format(knn_evalue.rmse()))
print('svd rmse+ {}'.format(svd_evalue.rmse()))


RMSE: 0.9793
knn rmse+ 0.9793299211533906
RMSE: 0.7191
svd rmse+ 0.7190771856550636


extract the first and the last user together with their 10 nearest neighbor

In [9]:

first_uid = coll_recommend.surprice_training.build_full_trainset().\
    to_inner_uid(list(coll_recommend.test_data.sort_values(
        by='unixReviewTime')['reviewerID'])[0])
last_uid = coll_recommend.surprice_training.build_full_trainset().\
    to_inner_uid(list(coll_recommend.test_data.sort_values(
        by='unixReviewTime')['reviewerID'])[-1])
nnfor_fisrt = [list(coll_recommend.test_data.sort_values(
    by='unixReviewTime')['reviewerID'])[0]]+[coll_recommend.surprice_training.build_full_trainset().
                                             to_raw_uid(i) for i in knn_model.get_neighbors(first_uid, 10)]
nnfor_last = [list(coll_recommend.test_data.sort_values(
    by='unixReviewTime')['reviewerID'])[-1]]+[coll_recommend.surprice_training.build_full_trainset().
                                              to_raw_uid(i) for i in knn_model.get_neighbors(last_uid, 10)]


customize a new user item matrix with first/last user and it's neighbor

In [10]:
first_user_nn10_matrix = coll_recommend.customer_ui_matrix(
    user_list=nnfor_fisrt)
last_user_nn10_matrix = coll_recommend.customer_ui_matrix(user_list=nnfor_last)


print their rating history

In [11]:
print(first_user_nn10_matrix)


asin           B005S4Y65I B009CCVMO0 B00F8K9KZS B00BR082FW B000UJUJ7U  \
A2G0O4Y8QE10AE        NaN        NaN        NaN        NaN        NaN   
A100UD67AHFODS          5          5          5          5        NaN   
A105S56ODHGJEK        NaN        NaN        NaN        NaN          3   
A1075X1Q4M3S78        NaN        NaN        NaN        NaN        NaN   
A10C5CJK1YKGV0        NaN        NaN        NaN        NaN        NaN   
A10CRW7XRJBJ2G        NaN        NaN        NaN        NaN        NaN   
A10EIJM2C94M14        NaN        NaN        NaN        NaN        NaN   
A10G3LHNAK4GEH        NaN        NaN        NaN        NaN        NaN   
A10G4BPT5MGBHY        NaN        NaN        NaN        NaN        NaN   
A10GU5NVTA5I67        NaN        NaN        NaN        NaN        NaN   
A10NC6ZVVMBHNH        NaN        NaN        NaN        NaN        NaN   

asin           B0013O54OE B001AFFZM2 B001EJQTUG B001UO8M8M B001W2Y5Q0  ...  \
A2G0O4Y8QE10AE        NaN        NaN        N

In [12]:
print(last_user_nn10_matrix)


asin           B005S4Y65I B009CCVMO0 B00F8K9KZS B00BR082FW B000UJUJ7U  \
A2SACTIFMC5DXO        NaN        NaN        NaN        NaN        NaN   
A23E9QQHJLNGUI        NaN        NaN        NaN        NaN        NaN   
A2DV14M0BC6YY1        NaN        NaN        NaN        NaN        NaN   
A100UD67AHFODS          5          5          5          5        NaN   
A105S56ODHGJEK        NaN        NaN        NaN        NaN          3   
A1075X1Q4M3S78        NaN        NaN        NaN        NaN        NaN   
A10C5CJK1YKGV0        NaN        NaN        NaN        NaN        NaN   
A10CRW7XRJBJ2G        NaN        NaN        NaN        NaN        NaN   
A10EIJM2C94M14        NaN        NaN        NaN        NaN        NaN   
A10G3LHNAK4GEH        NaN        NaN        NaN        NaN        NaN   
A10G4BPT5MGBHY        NaN        NaN        NaN        NaN        NaN   

asin           B0013O54OE B001AFFZM2 B001EJQTUG B001UO8M8M B001W2Y5Q0  ...  \
A2SACTIFMC5DXO        NaN        NaN        N

full fill Nan with 0 or the row mean value

In [26]:
fun10_np = first_user_nn10_matrix.to_numpy(dtype='float')
reset_fun10_np = []
for i in range(len(fun10_np)):
    line = np.array(fun10_np[i, :], dtype='float')
    line[np.isnan(line)] = np.nanmean(line)
    reset_fun10_np.append(line)
for i in range(len(fun10_np)):
    line = fun10_np[i, :]
    line[np.isnan(line)] = 0
reset_fun10_cs = cosine_similarity(reset_fun10_np[1:], [reset_fun10_np[0]])
fun10_cs = cosine_similarity(fun10_np[1:], [fun10_np[0]])
lun10_np = last_user_nn10_matrix.to_numpy(dtype='float')
reset_lun10_np = []
for i in range(len(lun10_np)):
    line = np.array(lun10_np[i, :], dtype='float')
    line[np.isnan(line)] = np.nanmean(line)
    reset_lun10_np.append(line)
for i in range(len(lun10_np)):
    line = lun10_np[i, :]
    line[np.isnan(line)] = 0
reset_lun10_cs = cosine_similarity(reset_lun10_np[1:], [reset_lun10_np[0]])
lun10_cs = cosine_similarity(lun10_np[1:], [lun10_np[0]])


cosine similarity within first/last user and their neighbors , filling Nan with 0

In [24]:
print(fun10_cs)
print(lun10_cs)


[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.2116083 ]
 [0.19762338]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]


cosine similarity within first/last user and their neighbors , filling Nan with row mean

In [25]:
print(reset_fun10_cs)
print(reset_lun10_cs)


[[1.        ]
 [0.99779248]
 [1.        ]
 [0.99945357]
 [0.99958359]
 [1.        ]
 [0.99994213]
 [0.99931194]
 [0.99997923]
 [0.999745  ]]
[[0.9981022 ]
 [0.99836939]
 [0.99967762]
 [0.99747081]
 [0.99967762]
 [0.99913137]
 [0.99926135]
 [0.99967762]
 [0.99961978]
 [0.99898978]]


form a prediction dictionary for knn model, with the structure {uid:(sorted prediction result)} 

later used to calculate the RR score

In [42]:
prediction_dic = knn_evalue.rank_k()

RR value for first user and his 10 neighbor

In [43]:
for uid in nnfor_fisrt:
    print(uid, ' rr+ ',
          knn_evalue.rr_k_user(prediction_dic[uid], uid, len(prediction_dic[uid])))


A2G0O4Y8QE10AE  rr+  0.001594896331738437
A100UD67AHFODS  rr+  0.004608294930875576
A105S56ODHGJEK  rr+  0.002898550724637681
A1075X1Q4M3S78  rr+  0.02040816326530612
A10C5CJK1YKGV0  rr+  0.0022123893805309734
A10CRW7XRJBJ2G  rr+  0.0025575447570332483
A10EIJM2C94M14  rr+  0.0027100271002710027
A10G3LHNAK4GEH  rr+  0.02127659574468085
A10G4BPT5MGBHY  rr+  0.0024813895781637717
A10GU5NVTA5I67  rr+  0.0038314176245210726
A10NC6ZVVMBHNH  rr+  0.0013315579227696406


RR value for first user and his 10 neighbor

In [44]:
for uid in nnfor_last:
    print(uid, ' rr+ ',
          knn_evalue.rr_k_user(prediction_dic[uid], uid, len(prediction_dic[uid])))


A2SACTIFMC5DXO  rr+  0.009433962264150943
A23E9QQHJLNGUI  rr+  0.0014814814814814814
A2DV14M0BC6YY1  rr+  0
A100UD67AHFODS  rr+  0.004608294930875576
A105S56ODHGJEK  rr+  0.002898550724637681
A1075X1Q4M3S78  rr+  0.02040816326530612
A10C5CJK1YKGV0  rr+  0.0022123893805309734
A10CRW7XRJBJ2G  rr+  0.0025575447570332483
A10EIJM2C94M14  rr+  0.0027100271002710027
A10G3LHNAK4GEH  rr+  0.02127659574468085
A10G4BPT5MGBHY  rr+  0.0024813895781637717


## Part4 Text Representation

initial the class for Contentbased recommendation, for different text representations I use different initial classes to record it

In [9]:
tfidf_model = msy.Contentbased_recommendation(reuse=preprocess)
word2vec_model = msy.Contentbased_recommendation(reuse=preprocess,word2vec=True)
bert_model = msy.Contentbased_recommendation(reuse=preprocess,bert=True)


after tokenizing and other processing the remaining word size

In [46]:
tfidf_model.print_words_num()


622


the size of tfidf item profile

In [47]:
print(tfidf_model.tfidf.to_numpy().shape)


(801, 618)


similarity between products within the TF-IDF, word2vec and BERT

In [48]:
print('tfidf', ' + ',
      cosine_similarity(tfidf_model.tfidf.iloc[:5, :].to_numpy()))
print('word2vec', ' + ',
      cosine_similarity(word2vec_model.tfidf.iloc[:5, :].to_numpy()))
print('bert', ' + ',
      cosine_similarity(bert_model.tfidf.iloc[:5, :].to_numpy()))


tfidf  +  [[1.         0.40876432 0.38100113 0.         0.        ]
 [0.40876432 1.         0.66379832 0.         0.        ]
 [0.38100113 0.66379832 1.         0.         0.        ]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         0.         0.         1.        ]]
word2vec  +  [[1.         0.49886109 0.48628955 0.64383875 0.3779223 ]
 [0.49886109 1.         0.82148934 0.40978907 0.32713615]
 [0.48628955 0.82148934 1.         0.43663537 0.3267278 ]
 [0.64383875 0.40978907 0.43663537 1.         0.41686391]
 [0.3779223  0.32713615 0.3267278  0.41686391 1.        ]]
bert  +  [[1.         0.82374993 0.83612338 0.61856529 0.64412636]
 [0.82374993 1.         0.94925152 0.66393282 0.62677793]
 [0.83612338 0.94925152 1.         0.67859027 0.67339287]
 [0.61856529 0.66393282 0.67859027 1.         0.70500187]
 [0.64412636 0.62677793 0.67339287 0.70500187 1.        ]]


## Part 5 Content-Based Recommender System


making prediction with TF-IDF, word2vec and BERT

In [10]:
tfidf_prediction = tfidf_model.predict()
word2vec_prediction = word2vec_model.predict()
bert_prediction = bert_model.predict()


load the class for making the evaluation for the predictions

In [11]:
tfidf_evalue = msy.Evaluation(tfidf_prediction, reuse=preprocess)
word2vec_evalue = msy.Evaluation(word2vec_prediction, reuse=preprocess)
bert_evalue = msy.Evaluation(bert_prediction, reuse=preprocess)


evaluation for CB system with TF-IDF

In [51]:
print('tfidf mpk+  {:.6f}'.format(tfidf_evalue.mean_k(5, 'p_k')))
print('tfidf mapk+ {:.6f}'.format(tfidf_evalue.mean_k(5, 'ap_k')))
print('tfidf mrrk+ {:.6f}'.format(tfidf_evalue.mean_k(5, 'rr_k')))
print('tfidf hr+   {:.6f}'.format(tfidf_evalue.mean_k(5, 'hr_k')))


tfidf mpk+ 0.041613
tfidf mapk+ 0.162293
tfidf mrrk+ 0.162293
tfidf hr+ 0.208065


evaluation for CB system with word2vec

In [52]:
print('word2vec mpk+  {:.6f}'.format(word2vec_evalue.mean_k(5, 'p_k')))
print('word2vec mapk+ {:.6f}'.format(word2vec_evalue.mean_k(5, 'ap_k')))
print('word2vec mrrk+ {:.6f}'.format(word2vec_evalue.mean_k(5, 'rr_k')))
print('word2vec hr+   {:.6f}'.format(word2vec_evalue.mean_k(5, 'hr_k')))


word2vec mpk+ 0.035067
word2vec mapk+ 0.147078
word2vec mrrk+ 0.147078
word2vec hr+ 0.175336


evaluation for CB system with BERT

In [53]:
print('bert mpk+  {:.6f}'.format(bert_evalue.mean_k(5, 'p_k')))
print('bert mapk+ {:.6f}'.format(bert_evalue.mean_k(5, 'ap_k')))
print('bert mrrk+ {:.6f}'.format(bert_evalue.mean_k(5, 'rr_k')))
print('bert hr+   {:.6f}'.format(bert_evalue.mean_k(5, 'hr_k')))


bert mpk+ 0.035885
bert mapk+ 0.061368
bert mrrk+ 0.061368
bert hr+ 0.179427


## Part 6 Hybrid Recommender System

combine two evaluation classes to form a evaluation list which is used in fit the Hybrid recommender system class 

in fact you are free to use more than 2 models to form this but I didn't do the test 

In [36]:
evaluation_list = [svd_evalue, tfidf_evalue]


Initial a Hybrid recommender system class with svd and tfidf model, for Meta-level strategy the user profile comes from tfidf model

In [37]:
hybrid_model = msy.Hybridrecommender_system(
    evaluation_list=evaluation_list, reuse=preprocess, userprofile=tfidf_model.user_mean_vector)


run weight strategy with weight [1,1] [1,2] and [2,1]

In [14]:
ws_pred_11 = hybrid_model.weight_strategy([1, 1])
ws_pred_12 = hybrid_model.weight_strategy([1, 2])
ws_pred_21 = hybrid_model.weight_strategy([2, 1])


initial evaluation class for them

In [15]:
ws11_evalue = msy.Evaluation(ws_pred_11, reuse=preprocess)
ws12_evalue = msy.Evaluation(ws_pred_12, reuse=preprocess)
ws21_evalue = msy.Evaluation(ws_pred_21, reuse=preprocess)


MP@k, MAP@k, MRR@k, MHR@k for weight strategy with weight [1,1]

In [16]:
print(
    'ws [svd tfidf] mpk+  {:.6f} with weight [1,1]'.format(ws11_evalue.mean_k(5, 'p_k')))
print(
    'ws [svd tfidf] mapk+ {:.6f} with weight [1,1]'.format(ws11_evalue.mean_k(5, 'ap_k')))
print(
    'ws [svd tfidf] mrrk+ {:.6f} with weight [1,1]'.format(ws11_evalue.mean_k(5, 'rr_k')))
print(
    'ws [svd tfidf] hr+   {:.6f} with weight [1,1]'.format(ws11_evalue.mean_k(5, 'hr_k')))


ws [svd tfidf] mpk+  0.041847 with weight [1,1]
ws [svd tfidf] mapk+ 0.161601 with weight [1,1]
ws [svd tfidf] mrrk+ 0.161601 with weight [1,1]
ws [svd tfidf] hr+   0.209234 with weight [1,1]


MP@k, MAP@k, MRR@k, MHR@k for weight strategy with weight [1,2]

In [60]:
print(
    'ws [svd tfidf] mpk+  {:.6f} with weight [1,2]'.format(ws12_evalue.mean_k(5, 'p_k')))
print(
    'ws [svd tfidf] mapk+ {:.6f} with weight [1,2]'.format(ws12_evalue.mean_k(5, 'ap_k')))
print(
    'ws [svd tfidf] mrrk+ {:.6f} with weight [1,2]'.format(ws12_evalue.mean_k(5, 'rr_k')))
print(
    'ws [svd tfidf] hr+   {:.6f} with weight [1,2]'.format(ws12_evalue.mean_k(5, 'hr_k')))


ws mpk+ 0.042548 with weight [1,2]
ws mapk+ 0.164757 with weight [1,2]
ws mrrk+ 0.164757 with weight [1,2]
ws hr+ 0.212741 with weight [1,2]


MP@k, MAP@k, MRR@k, MHR@k for weight strategy with weight [2,1]

In [61]:
print(
    'ws [svd tfidf] mpk+  {:.6f} with weight [2,1]'.format(ws21_evalue.mean_k(5, 'p_k')))
print(
    'ws [svd tfidf] mapk+ {:.6f} with weight [2,1]'.format(ws21_evalue.mean_k(5, 'ap_k')))
print(
    'ws [svd tfidf] mrrk+ {:.6f} with weight [2,1]'.format(ws21_evalue.mean_k(5, 'rr_k')))
print(
    'ws [svd tfidf] hr+   {:.6f} with weight [2,1]'.format(ws21_evalue.mean_k(5, 'hr_k')))


ws mpk+ 0.038574 with weight [2,1]
ws mapk+ 0.157423 with weight [2,1]
ws mrrk+ 0.157423 with weight [2,1]
ws hr+ 0.192870 with weight [2,1]


weight strategy with Borda Fuse score method

In [17]:
bf_model = hybrid_model.weight_strategy(borda=True, rank=100)


initial the evaluation class for bf method output

In [18]:
bf_evalue = msy.Evaluation(bf_model, reuse=preprocess)


MP@k, MAP@k, MRR@k, MHR@k for weight strategy with Borda Fuse

In [19]:
print('bf ws [svd tfidf] mpk+  {:.6f}'.format(bf_evalue.mean_k(5, 'p_k')))
print('bf ws [svd tfidf] mapk+ {:.6f}'.format(bf_evalue.mean_k(5, 'ap_k')))
print('bf ws [svd tfidf] mrrk+ {:.6f}'.format(bf_evalue.mean_k(5, 'rr_k')))
print('bf ws [svd tfidf] hr+   {:.6f}'.format(bf_evalue.mean_k(5, 'hr_k')))


bf ws [svd tfidf] mpk+  0.031911
bf ws [svd tfidf] mapk+ 0.143590
bf ws [svd tfidf] mrrk+ 0.143590
bf ws [svd tfidf] hr+   0.159556


make prediction with switch strategy 

In [20]:
ss_pred = hybrid_model.switching_strategy()


initial evaluation

In [21]:
ss_evalue = msy.Evaluation(ss_pred, reuse=preprocess)


MP@k, MAP@k, MRR@k, MHR@k for switch strategy

In [22]:
print('ss [svd tfidf] mpk+  {:.6f}'.format(ss_evalue.mean_k(5, 'p_k')))
print('ss [svd tfidf] mapk+ {:.6f}'.format(ss_evalue.mean_k(5, 'ap_k')))
print('ss [svd tfidf] mrrk+ {:.6f}'.format(ss_evalue.mean_k(5, 'rr_k')))
print('ss [svd tfidf] hr+   {:.6f}'.format(ss_evalue.mean_k(5, 'hr_k')))


ss [svd tfidf] mpk+  0.042314
ss [svd tfidf] mapk+ 0.164144
ss [svd tfidf] mrrk+ 0.164144
ss [svd tfidf] hr+   0.211572


predict with Meta-level strategy and take the 5 nearest neighbors

In [38]:
ms_pred = hybrid_model.metalevel_strategy(rank=5)


initial evaluation

In [41]:
ms_evalue = msy.Evaluation(ms_pred, reuse=preprocess)


MP@k, MAP@k, MRR@k, MHR@k for Meta-level strategy

In [42]:
print('ms [svd tfidf] mpk+  {:.6f}'.format(ms_evalue.mean_k(5, 'p_k')))
print('ms [svd tfidf] mapk+ {:.6f}'.format(ms_evalue.mean_k(5, 'ap_k')))
print('ms [svd tfidf] mrrk+ {:.6f}'.format(ms_evalue.mean_k(5, 'rr_k')))
print('ms [svd tfidf] hr+   {:.6f}'.format(ms_evalue.mean_k(5, 'hr_k')))


ms [svd tfidf] mpk+  0.019871
ms [svd tfidf] mapk+ 0.049786
ms [svd tfidf] mrrk+ 0.049786
ms [svd tfidf] hr+   0.099357


Initial a hybrid recommender system class with [SVD, TF-IDF] and using the prediction from TF-IDF to full fill the user item matrix, scale score from TF-IDF to [0,5] and make the prediction with parameter rank=5, also initial the evaluation class for it.

<font color=red>Warning</font> the step takes about 8 minutes 


In [43]:
hybrid_model_withpred = msy.Hybridrecommender_system(
    evaluation_list=evaluation_list, reuse=preprocess, userprofile=tfidf_model.user_mean_vector, with_pred=1, scalar=5)
ms_pred_withpred = hybrid_model_withpred.metalevel_strategy(rank=5)
ms_evalue_withpred = msy.Evaluation(ms_pred_withpred, reuse=preprocess)


In [45]:
print('ms [svd tfidf] mpk+  {:.6f} with TF-IDF predction'.format(
    ms_evalue_withpred.mean_k(5, 'p_k')))
print('ms [svd tfidf] mapk+ {:.6f} with TF-IDF predction'.format(
    ms_evalue_withpred.mean_k(5, 'ap_k')))
print('ms [svd tfidf] mrrk+ {:.6f} with TF-IDF predction'.format(
    ms_evalue_withpred.mean_k(5, 'rr_k')))
print('ms [svd tfidf] hr+   {:.6f} with TF-IDF predction'.format(
    ms_evalue_withpred.mean_k(5, 'hr_k')))


ms [svd tfidf] mpk+  0.041964 with TF-IDF predction
ms [svd tfidf] mapk+ 0.168966 with TF-IDF predction
ms [svd tfidf] mrrk+ 0.168966 with TF-IDF predction
ms [svd tfidf] hr+   0.209819 with TF-IDF predction


using KNN and TF-IDF evaluation class to form a new list

In [23]:
evaluation_list = [knn_evalue, tfidf_evalue]


initial a Hybrid recommender system with [KNN, TF-IDF]

In [24]:
hybrid_model_knn = msy.Hybridrecommender_system(
    evaluation_list=evaluation_list, reuse=preprocess, userprofile=tfidf_model.user_mean_vector)


predict with weight strategy and the weight is [1,1]

In [25]:
ws_pred_11_knn = hybrid_model_knn.weight_strategy([1, 1])
ws11_evalue_knn = msy.Evaluation(ws_pred_11_knn, reuse=preprocess)


MP@k, MAP@k, MRR@k, MHR@k for weight strategy with weight [1,1] under [KNN, TF-IDF] pair

In [26]:
print(
    'ws [knn tfidf]mpk+  {:.6f} with weight [1,1]'.format(ws11_evalue_knn.mean_k(5, 'p_k')))
print(
    'ws [knn tfidf]mapk+ {:.6f} with weight [1,1]'.format(ws11_evalue_knn.mean_k(5, 'ap_k')))
print(
    'ws [knn tfidf]mrrk+ {:.6f} with weight [1,1]'.format(ws11_evalue_knn.mean_k(5, 'rr_k')))
print(
    'ws [knn tfidf]hr+   {:.6f} with weight [1,1]'.format(ws11_evalue_knn.mean_k(5, 'hr_k')))


ws [knn tfidf]mpk+  0.007481 with weight [1,1]
ws [knn tfidf]mapk+ 0.018576 with weight [1,1]
ws [knn tfidf]mrrk+ 0.018576 with weight [1,1]
ws [knn tfidf]hr+   0.037405 with weight [1,1]


predict with switch strategy

In [54]:
ss_pred_knn = hybrid_model_knn.switching_strategy()
ss_evalue_knn = msy.Evaluation(ss_pred_knn, reuse=preprocess)


MP@k, MAP@k, MRR@k, MHR@k for switch strategy under [KNN, TF-IDF] pair

In [55]:
print('ss [knn tfidf]mpk+  {:.6f}'.format(ss_evalue_knn.mean_k(5, 'p_k')))
print('ss [knn tfidf]mapk+ {:.6f}'.format(ss_evalue_knn.mean_k(5, 'ap_k')))
print('ss [knn tfidf]mrrk+ {:.6f}'.format(ss_evalue_knn.mean_k(5, 'rr_k')))
print('ss [knn tfidf]hr+   {:.6f}'.format(ss_evalue_knn.mean_k(5, 'hr_k')))


ss [knn tfidf]mpk+  0.044886
ss [knn tfidf]mapk+ 0.170505
ss [knn tfidf]mrrk+ 0.170505
ss [knn tfidf]hr+   0.224430


with cut_off = 100 initial a Hybrid recommender system with [SVD, TF-IDF], run weight strategy with [1,2] and switch strategy

In [27]:
evaluation_list = [svd_evalue, tfidf_evalue]
hybrid_model_withcutoff = msy.Hybridrecommender_system(
    evaluation_list, reuse=preprocess, cut_off=100)
ws_pred_12_withcutoff = hybrid_model_withcutoff.weight_strategy([1, 2])
ss_pred_withcutoff = hybrid_model_withcutoff.switching_strategy()


initial evaluation

In [28]:
ws_evalue_12_withcutoff = msy.Evaluation(
    ws_pred_12_withcutoff, reuse=preprocess)
ss_evalue_withcutoff = msy.Evaluation(ss_pred_withcutoff, reuse=preprocess)


MP@k, MAP@k, MRR@k, MHR@k for weight strategy with cut-off=100 and weight [1,2]

In [29]:
print('ws [svd tfidf] mpk+  {:.6f} with weight [1,2] with cutoff'.format(
    ws_evalue_12_withcutoff.mean_k(5, 'p_k')))
print('ws [svd tfidf] mapk+ {:.6f} with weight [1,2] with cutoff'.format(
    ws_evalue_12_withcutoff.mean_k(5, 'ap_k')))
print('ws [svd tfidf] mrrk+ {:.6f} with weight [1,2] with cutoff'.format(
    ws_evalue_12_withcutoff.mean_k(5, 'rr_k')))
print('ws [svd tfidf] hr+   {:.6f} with weight [1,2] with cutoff'.format(
    ws_evalue_12_withcutoff.mean_k(5, 'hr_k')))


ws [svd tfidf] mpk+  0.038925 with weight [1,2] with cutoff
ws [svd tfidf] mapk+ 0.156750 with weight [1,2] with cutoff
ws [svd tfidf] mrrk+ 0.156750 with weight [1,2] with cutoff
ws [svd tfidf] hr+   0.194623 with weight [1,2] with cutoff


MP@k, MAP@k, MRR@k, MHR@k for switch strategy with cut-off=100

In [30]:
print(
    'ss [svd tfidf] mpk+  {:.6f} with cutoff'.format(ss_evalue_withcutoff.mean_k(5, 'p_k')))
print(
    'ss [svd tfidf] mapk+ {:.6f} with cutoff'.format(ss_evalue_withcutoff.mean_k(5, 'ap_k')))
print(
    'ss [svd tfidf] mrrk+ {:.6f} with cutoff'.format(ss_evalue_withcutoff.mean_k(5, 'rr_k')))
print(
    'ss [svd tfidf] hr+   {:.6f} with cutoff'.format(ss_evalue_withcutoff.mean_k(5, 'hr_k')))


ss [svd tfidf] mpk+  0.042314 with cutoff
ss [svd tfidf] mapk+ 0.164144 with cutoff
ss [svd tfidf] mrrk+ 0.164144 with cutoff
ss [svd tfidf] hr+   0.211572 with cutoff
